In [1]:
from glob import glob

In [52]:
r = glob('rationales/FT3/batch_*')
print(len(r))

14171


In [58]:
path = r[0]
path[path.find('_')+1:]

'13747'

In [6]:
print(len(glob('outputs/FT3/attentions/batch_*.pt')))

14345


In [2]:
2

2

In [3]:
from datasets import load_from_disk
batch = load_from_disk('kr4/FT3/batch_9')

In [4]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
def tokenize_func(example):
    return tokenizer(example['Rationale'], truncation=True)

In [5]:
# tokenize
batch = batch.map(tokenize_func, batched=True)

# process columns
batch = batch.remove_columns(['Unrationale', 'Rationale'])
batch = batch.rename_column('Rating', 'labels')
batch.set_format(type='torch')

Loading cached processed dataset at kr4/FT3/batch_9/cache-36e05c2708720c3f.arrow


In [9]:
from glob import glob
paths = glob(f'kr4/FT3/batch_*')

In [13]:
paths[0][paths[0].find('_')+1:]

'13747'

In [14]:
tokenized = load_from_disk('kr4_tokenized/FT3/batch_0')

In [15]:
tokenized

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 32
})

## PyTorch

In [16]:
from torch.utils.data import Dataset, DataLoader
from glob import glob

kr3_tokenized

In [61]:
from datasets import Dataset
kr3_tokenized = Dataset.load_from_disk('kr3_tokenized_maxpad')

In [62]:
kr3_tokenized

Dataset({
    features: ['labels', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 459021
})

In [69]:
print(kr3_tokenized[0]['input_ids'][:10])
print(kr3_tokenized[1]['input_ids'][:10])

[101, 9461, 17138, 9096, 68833, 12310, 9665, 25934, 34907, 58303]
[101, 8888, 47869, 9670, 89523, 9254, 119192, 119138, 12965, 48549]


In [65]:
dloader = DataLoader(kr3_tokenized, batch_size=4)
next(iter(dloader))

{'labels': tensor([1, 1, 1, 1]),
 'input_ids': [tensor([101, 101, 101, 101]),
  tensor([9461, 8888, 9656, 8888]),
  tensor([17138, 47869, 31605, 12310]),
  tensor([9096, 9670, 9555, 9543]),
  tensor([68833, 89523, 11664, 10739]),
  tensor([12310,  9254,  8942,  9935]),
  tensor([  9665, 119192, 118707, 119231]),
  tensor([ 25934, 119138,    117,  70146]),
  tensor([34907, 12965,  9064,  9891]),
  tensor([58303, 48549, 63218,  9462]),
  tensor([48345,   106, 11102, 14423]),
  tensor([  119,   100,  9254, 20479]),
  tensor([  8865,   8843,  10459, 118969]),
  tensor([ 29364, 118813,   9462,  10622]),
  tensor([  9283,  90587, 118781,   9485]),
  tensor([ 37114,   9004,  20479, 119330]),
  tensor([20729, 32537, 10739,  8932]),
  tensor([ 9258,  9685,  9462, 37712]),
  tensor([ 13890, 119118, 118781,  10739]),
  tensor([ 9283, 16985, 20479,  9119]),
  tensor([37114, 48549, 10622, 48345]),
  tensor([12092,  9641,  9521,  9254]),
  tensor([ 9682, 35506,  9685, 12092]),
  tensor([ 9891, 14040

kr4

In [1]:
from transformers import BertForSequenceClassification
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

In [11]:
from datasets import load_from_disk
batch = load_from_disk('kr4_tokenized/FT3_TopK/batch_0')

In [15]:
import torch
torch.tensor(batch['labels'])

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1,
        1, 0, 1, 0, 1, 1, 1, 1])

In [5]:
model.eval()
with torch.no_grad():
    out = model(**tensor_batch)

In [19]:
from glob import glob
paths = glob('kr4_tokenized/FT3/batch_*')

In [20]:
train_paths = paths[ : int(len(paths)*.7)]
val_paths = paths[int(len(paths)*.7) : int(len(paths)*.8)]
test_paths = paths[int(len(paths)*.8) : ]

In [21]:
len(train_paths), len(val_paths), len(test_paths)

(9919, 1417, 2835)

## TorchMetrics

In [7]:
from torchmetrics import MetricCollection, F1Score
import torch

In [4]:
f1score = F1Score(num_classes=2, average='none')

In [9]:
for _ in range(5):
    f1score.update(torch.tensor([1,0,0,0]), torch.tensor([0,0,1,0]))

In [10]:
f1score.compute()

tensor([0.6667, 0.0000])

## Large batch

In [21]:
from datasets import load_from_disk
from glob import glob

In [47]:
paths = glob(f'kr4_tokenized/FT3_TopK/batch_*') # behavior of glob is deterministic.
train_paths = paths[ : int(len(paths)*.8)]
path = train_paths[682]
batch = load_from_disk(path)
len(batch['input_ids'][1])


500

In [19]:
from tqdm import tqdm
for i in tqdm(range(4)):
    pass

100%|██████████| 4/4 [00:00<00:00, 44979.13it/s]
